In [1]:
pip install networkx matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: C:\Users\dimeg\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
from pyswip import Prolog
import sys
import os
import webbrowser 
import urllib.parse 

# CONFIGURACION Y DATOS 

# Diccionario de Agencias (DATOS REALES)
AGENCIAS = {
    # --- CAPITAL FEDERAL ---
    'caba1': {'nombre': 'Kansai (Caballito)', 'zona': 'CABA', 'direccion': 'Av. Rivadavia 6102, C1406 Cdad. Autónoma de Buenos Aires'},
    'caba2': {'nombre': 'Ford Dietrich', 'zona': 'CABA', 'direccion': 'Av. Corrientes 4242, C1195 Cdad. Autónoma de Buenos Aires'},
    'caba3': {'nombre': 'Volkswagen Alra', 'zona': 'CABA', 'direccion': 'Av. Corrientes 5974, C1414AJM Cdad. Autónoma de Buenos Aires'},
    'caba4': {'nombre': 'Espasa VW (Pto Madero)', 'zona': 'CABA', 'direccion': 'Av. Paseo Colón 494, C1063ACR Cdad. Autónoma de Buenos Aires'},
    'caba5': {'nombre': 'Collins Chevrolet', 'zona': 'CABA', 'direccion': 'Av. Francisco Beiró 2955, C1419 Cdad. Autónoma de Buenos Aires'},
    'caba6': {'nombre': 'Mapemfi', 'zona': 'CABA', 'direccion': 'Av. Belgrano 1602, C1093AAR Cdad. Autónoma de Buenos Aires'},
    
    # --- ZONA NORTE ---
    'norte1': {'nombre': 'Car One (Tortuguitas)', 'zona': 'Norte', 'direccion': 'Panamericana, Ramal Pilar km 35.5, B1667 Tortuguitas, Provincia de Buenos Aires'},
    'norte2': {'nombre': 'Autodelta', 'zona': 'Norte', 'direccion': 'Av. del Sesquicentenario 2705, B1613 Los Polvorines, Provincia de Buenos Aires'},
    'norte3': {'nombre': 'Audi Norden', 'zona': 'Norte', 'direccion': 'Av. del Libertador 15154, B1640 APB, Provincia de Buenos Aires'},

    # --- ZONA SUR ---
    'sur1': {'nombre': 'Strianese Ford', 'zona': 'Sur', 'direccion': 'Av. Hipólito Yrigoyen 8679, B1832 Lomas de Zamora, Provincia de Buenos Aires'},
    'sur2': {'nombre': 'Autos del Sur (Quilmes)', 'zona': 'Sur', 'direccion': 'Av. Hipólito Yrigoyen 80, B1878 Quilmes, Provincia de Buenos Aires'},
    'sur3': {'nombre': 'Del Sur Autos (Lanús)', 'zona': 'Sur', 'direccion': 'Av. Hipólito Yrigoyen 3365, B1824 Lanús, Provincia de Buenos Aires'},

    # --- ZONA OESTE ---
    'oeste1': {'nombre': 'Del Oeste (San Justo)', 'zona': 'Oeste', 'direccion': 'Dr. Enrique Eizaguirre 2560, B1754FTQ San Justo, Provincia de Buenos Aires'},
    'oeste2': {'nombre': 'Vehículos Del Oeste', 'zona': 'Oeste', 'direccion': 'Salta 3506, B1754IRL San Justo, Provincia de Buenos Aires'},
    'oeste3': {'nombre': 'LNG Olivieri', 'zona': 'Oeste', 'direccion': 'Av. Rivadavia 12980, B1702 Ciudadela, Provincia de Buenos Aires'},
}

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

prolog = Prolog()
archivo_prolog = "carbotdef.pl"
archivo_csv = "Autosdef.csv" 

if not os.path.exists(archivo_prolog):
    print(f"❌ ERROR: No está {archivo_prolog}")
    sys.exit()

# Cargar Prolog
prolog.consult(archivo_prolog)

# FUNCIONES AUXILIARES

def cargar_datos():
    print(f"📂 Cargando base de conocimientos...")
    try:
        df = pd.read_csv(archivo_csv, sep=';')
        if len(df.columns) < 2: df = pd.read_csv(archivo_csv, sep=',')
        df.columns = df.columns.str.strip().str.lower()
    except:
        print("❌ Error leyendo CSV. Verifica el archivo."); sys.exit()

    count = 0
    
    stats = {}
    
    for index, row in df.iterrows():
        try:
            m = str(row['marca']).lower().strip()
            mo = str(row['modelo']).lower().strip()
            v = str(row['version']).lower().strip().replace("'","")
            
            t = str(row['tipo']).lower().strip()

            val_anio = str(row['anio']).replace('.','').replace(',','')
            a = int(float(val_anio))
            
            val_precio = str(row['precio']).replace('.','').replace(',','')
            p = int(float(val_precio))
            
            prolog.assertz(f"auto('{m}','{mo}','{t}','{v}',{a},{p})")
            
            count += 1
            if t in stats: stats[t] += 1
            else: stats[t] = 1
            
        except: continue
    
    print(f"✅ Sistema Experto online: {count} vehículos.")
    print(f"📊 Clasificación CSV: {stats}")

def obtener_estado():
    r = list(prolog.query("estado_chat(E)"))
    return r[0]['E'] if r else 'error'

def transicion(e):
    list(prolog.query(f"avanzar_estado({e})"))

def abrir_ruta_google_maps(ruta_codigos):
    print("\n🌎 Generando enlace de navegación GPS...")
    direcciones = []
    
    for codigo in ruta_codigos:
        clave = str(codigo)
        if clave in AGENCIAS:
            dir_texto = AGENCIAS[clave]['direccion']
            direcciones.append(urllib.parse.quote(dir_texto))
    
    if not direcciones:
        print("❌ Error: No hay direcciones para mapear.")
        return

    base_url = "https://www.google.com/maps/dir/"
    url_final = base_url + "/".join(direcciones)
    
    print(f"🔗 Link generado: {url_final}")
    print("🚀 Abriendo Google Maps en tu navegador...")
    webbrowser.open(url_final)


# PROGRAMA PRINCIPAL

def main():
    cargar_datos()
    print("\n=================================================")
    print("                 🚗 CARBOT 🚗")
    print("=================================================")
    print(" Sistema Experto | Turing | Automatas | TSP (GPS)")
    print(" Escribe 'salir' para terminar.\n")
    
    tipo_u = ""
    temp_min = 0 
    
    while True:
        est = obtener_estado()
        u_in = input(f"[{est.upper()}] Tú: ").strip().lower()
        if u_in == 'salir': break

        
        if est == "inicio":
            print("🤖 Bot: ¡Hola! Bienvenido a la Concesionaria CarBot.")
            print("   Selecciona una opción:")
            print("   [1] 🚗 Comprar un Auto (Ventas)")
            print("   [2] 🔧 Service / Taller (Post-Venta)")
            
            opcion = input("👉 Elige (1 o 2): ").strip()
            
            if opcion == '1':
                transicion("elegir_ventas")
            elif opcion == '2':
                transicion("elegir_taller")
                print("🤖 Bot: Bienvenido al Taller Oficial.")
                print("   ¿Qué necesitas realizar?")
                print("   [1] Service de los 10.000km / Garantía")
                print("   [2] Reparación General / Mecánica")
            else:
                print("⚠️ Opción no válida. Escribe 1 o 2.")

        
        elif est == "menu_taller":
            if u_in == '1':
                transicion("service_oficial")
                print("🔧 Agendando Service Oficial.")
                print("   Por favor, indica la patente de tu vehículo:")
            elif u_in == '2':
                transicion("reparacion")
                print("🔧 Agendando Revisión Mecánica.")
                print("   Por favor, indica la patente de tu vehículo:")
            else:
                 print("⚠️ Escribe 1 o 2.")

        elif est == "agendando_turno":
            # Limpia espacios y convierte a minusculas para Prolog
            patente_raw = u_in.replace(" ", "").lower()
            print(f"[GRAMÁTICA]: Validando estructura de patente '{patente_raw.upper()}'...")
            
            l_chars = list(patente_raw)
            es_valida = False
            
            try:
                # Consulta la Gramatica DCG
                if list(prolog.query(f"validar_patente({l_chars})")):
                    es_valida = True
            except: pass
            
            if es_valida:
                print(f"✅ Patente {patente_raw.upper()} válida.")
                print("   Un asesor de post-venta te contactará a la brevedad.")
                transicion("confirmar_turno") 
                print("🤖 Bot: ¡Gracias por confiar en nuestro servicio técnico!")
                transicion("finalizar") # Cierra el ciclo
            else:
                print("❌ PATENTE INVÁLIDA.")
                print("   Formatos aceptados:")
                print("   - Vieja: 3 Letras + 3 Números (Ej: ABC123)")
                print("   - Mercosur: 2 Letras + 3 Números + 2 Letras (Ej: AA123BB)")
                print("   Intenta de nuevo:")

        
        elif est == "preguntando_tipo":
            print("🤖 Bot: ¡Genial! Vamos a buscar tu próximo vehículo.")
            print("   Selecciona la categoría:")
            print("   [1] 🚗 Auto (Sedan / Hatchback / Coupe)")
            print("   [2] 🚙 SUV")
            print("   [3] 🛻 Pickup / Camioneta")
            print("   [4] 🚐 Utilitario / Furgón")
            
            sel = input("👉 Elige (1-4): ").strip()
            
            mapa_tipos = {'1':'auto', '2':'suv', '3':'pickup', '4':'utilitario'}
            
            if sel in mapa_tipos:
                tipo_u = mapa_tipos[sel]
                transicion("recibir_tipo")
                print(f"🤖 Bot: Entendido, buscas '{tipo_u.upper()}'.")
                print("   ¿Cuál es el precio MÍNIMO que quieres pagar (en USD)?")
            else:
                print("⚠️ Opción inválida. Por favor elige 1, 2, 3 o 4.")

        elif est == "preguntando_minimo":
            if u_in.isdigit():
                temp_min = int(u_in)
                transicion("recibir_min")
                print(f"🤖 Bot: Bien (Mín: ${temp_min}). ¿Y cuál es tu presupuesto MÁXIMO (en USD)?")
            else:
                print("🤖 Bot: Por favor, escribe un número entero.")

        elif est == "preguntando_maximo":
            if u_in.isdigit():
                max_val = int(u_in)
                if max_val < temp_min:
                    print(f"⚠️ El máximo debe ser mayor al mínimo.")
                    continue

                transicion("recibir_max") 
                print("🤖 Bot: Analizando mercado...")
                
                query = f"recomendar_rango('{tipo_u}', {temp_min}, {max_val}, L)"
                res = list(prolog.query(query))
                
                if res and res[0]['L']:
                    all_autos = res[0]['L']
                    all_autos.sort(key=lambda x: x[4])
                    total = len(all_autos)
                    
                    print(f"\n✅ Encontré {total} vehículos en tu rango (${temp_min} - ${max_val}).")
                    if total <= 15:
                         print("--- 📝 LISTADO COMPLETO ---")
                         for item in all_autos:
                            precio_fmt = "{:,}".format(item[4]).replace(",", ".")
                            print(f"   ${precio_fmt} | {item[0].upper()} {item[1].title()} {item[2]} ({item[3]})")
                    else:
                        baratos = all_autos[:5]
                        caros = all_autos[-5:]
                        caros.reverse()
                        print("--- 📉 TOP 5 OPORTUNIDADES ---")
                        for item in baratos:
                             precio_fmt = "{:,}".format(item[4]).replace(",", ".")
                             print(f"   ${precio_fmt} | {item[0].upper()} {item[1].title()} {item[2]} ({item[3]})")
                        print("\n--- 📈 TOP 5 ALTA GAMA (En Rango) ---")
                        for item in caros:
                             precio_fmt = "{:,}".format(item[4]).replace(",", ".")
                             print(f"   ${precio_fmt} | {item[0].upper()} {item[1].title()} {item[2]} ({item[3]})")

                    print("\n🤖 Bot: ¿Cómo te gustaría proceder?")
                    print("   [1] 💵 Me interesa / Ver formas de pago")
                    print("   [2] 🔄 Buscar de nuevo (No me convencen)")
                else:
                    print(f"🤖 Bot: No encontré nada. Intenta ampliar el rango.")
                    list(prolog.query("retract(estado_chat(_)), assert(estado_chat(inicio))"))
            else:
                 print("🤖 Bot: Por favor, escribe un número entero.")

        
        elif est == "recomendando":
            if u_in == '1' or u_in == 'si' or 'interesa' in u_in:
                transicion("interesar")
                print("\n-------------------------------------------------")
                print("💰  MÓDULO DE FINANCIACIÓN Y PAGOS  💰")
                print("-------------------------------------------------")
                print("🤖 Bot: Antes de coordinar el viaje, definamos el pago.")
                print("   [1] 💵 Contado (Precio Final con bonificación)")
                print("   [2] 📉 Plan de Ahorro / Cuotas (Financiado)")
                print("\n👉 ¿Cómo prefieres abonar? (Escribe 1 o 2)")

            elif u_in == '2' or u_in == 'no' or 'buscar' in u_in or 'volver' in u_in:
                # Bucle de busqueda: vuelve al estado de inicio
                transicion("no_convence")
                print("\n🔄 ¡Entendido! Reiniciemos la búsqueda.")
            
            else:
                print("⚠️ Elige 1 (Interesa) o 2 (Buscar de nuevo).")

        elif est == "eligiendo_pago":
            if u_in == '1' or 'contado' in u_in:
                transicion("pago_contado") # Pasa a 'preparando_ruta'
                print("\n✅ Excelente. Al ser CONTADO, congelamos el precio.")
                # Se borra el avance automatico para esperar confirmacion
                print("🤖 Bot: ¿Deseas realizar un recorrido por las concesionarias disponibles? (si/no)")

            elif u_in == '2' or 'cuotas' in u_in:
                transicion("pago_cuotas")
                print("\n📉 Plan de Financiación Flexible.")
                print("   [12] Plan Ahora 12 (Tasa 0%)")
                print("   [84] Plan Ahorro 70/30")
                print("\n👉 ¿Cuántas cuotas prefieres? (Escribe 12 u 84)")
            else:
                print("⚠️ Escribe 1 o 2.")

        elif est == "seleccionando_plazo":
            if u_in == '12' or u_in == '84':
                print(f"✅ Plan de {u_in} cuotas seleccionado.")
                transicion("confirmar_plazo") # Pasa a 'preparando_ruta'
                # Se borra el avance automatico para esperar confirmacion
                print("🤖 Bot: Plan registrado. ¿Deseas realizar un recorrido por las concesionarias disponibles? (si/no)")
            else:
                print("⚠️ Solo tenemos planes de 12 u 84 meses.")

        elif est == "preparando_ruta":
             if u_in == 'si':
                 transicion("iniciar_viaje") # Pasa a 'calculando_ruta'
                 
             elif u_in == 'no':
                 #Si dice no, avanza rapido los estados sin calcular nada para llegar directo a pedir el contacto
                 transicion("iniciar_viaje")  # Pasa a calculando
                 transicion("termino_ruta")   # Pasa directo a ofreciendo_info
                 print("🤖 Bot: Entendido, no haremos el viaje por ahora.")
                 print("\n🔔 ¿Te gustaría dejarnos tu celular para recibir novedades?")
                 print("   (Escribe 'si' para dejarlo, o 'no' para terminar)")
                 continue # Saltamos al siguiente ciclo para que procese 'ofreciendo_info'
        

        if est == "calculando_ruta" or (est == "preparando_ruta" and u_in == 'si'):
                
                print("🗺️  LOGÍSTICA DE RECORRIDO - AMBA Y CABA 🗺️")
                
                print("¿En qué zona estás? [1]CABA [2]Norte [3]Sur [4]Oeste")
                zona_elegida = ""
                while True:
                    z_in = input("👉 Elige (1-4): ").strip()
                    if z_in == '1': zona_elegida = 'CABA'; break
                    elif z_in == '2': zona_elegida = 'Norte'; break
                    elif z_in == '3': zona_elegida = 'Sur'; break
                    elif z_in == '4': zona_elegida = 'Oeste'; break
                    else: print("❌ Opción inválida.")

                print(f"\n📍 Agencias en {zona_elegida.upper()}:")
                agencias_locales = {k:v for k,v in AGENCIAS.items() if v['zona'] == zona_elegida}
                for k, v in agencias_locales.items(): print(f"   [{k}] {v['nombre']}")
                
                nodo_inicio = ""
                while True:
                    nodo_inicio = input("🏁 Código de partida (ej: caba1): ").strip().lower()
                    if nodo_inicio in agencias_locales: break
                    print("❌ Código inválido.")

                print("\n🌎 Otras agencias disponibles:")
                for z in ['CABA', 'Norte', 'Sur', 'Oeste']:
                    print(f"--- {z.upper()} ---")
                    for k, v in AGENCIAS.items():
                        if v['zona'] == z and k != nodo_inicio: print(f"   [{k}] {v['nombre']}")

                sel_destinos = input("\n👉 Destinos (separados por coma): ").lower()
                destinos_elegidos = [c.strip() for c in sel_destinos.split(',') if c.strip() in AGENCIAS and c.strip() != nodo_inicio]
                
                if not destinos_elegidos:
                    print("⚠️ Sin destinos. Usando predeterminado.")
                    if zona_elegida == 'CABA': destinos_elegidos = ['caba2']
                    else: destinos_elegidos = ['caba1']

                print(f"\n🤖 Bot: Calculando logística...")
                if len(destinos_elegidos) <= 2:
                    algoritmo = "ruta_optima"
                else:
                    algoritmo = "ruta_aproximada"

                destinos_str = "[" + ", ".join(destinos_elegidos) + "]"
                q = f"{algoritmo}({nodo_inicio}, {destinos_str}, Ruta, Costo)"
                
                res = list(prolog.query(q))
                
                if res:
                    ruta_raw = res[0]['Ruta']
                    costo = res[0]['Costo']
                    print(f"\n✅ RUTA SUGERIDA ({algoritmo}):")
                    for parada in ruta_raw:
                        nombre = AGENCIAS[str(parada)]['nombre']
                        print(f"   ⬇️  {nombre}")
                    print(f"⛽ Distancia Real Total: {costo} km")
                    abrir_ruta_google_maps(ruta_raw)
                
                transicion("termino_ruta") 
                print("\n🔔 ¿Te gustaría dejarnos tu celular?")
                print("   (Escribe 'si' para dejarlo, o 'no' para terminar)")

        elif est == "ofreciendo_info":
            if u_in == 'si':
                transicion("aceptar_info")
                print("\n📝 Por favor, ingresa tu número (AMBA, La Plata o Pilar).")
                print("   Ejemplos: 11 4444 5555  |  221 555 6666")
            else:
                transicion("rechazar_info")
                print("🤖 Bot: Entendido. ¡Que disfrutes tu búsqueda! 👋")

        elif est == "pedir_contacto":
            raw_num = u_in.replace(" ", "")
            print(f"[TURING]: Analizando sintaxis de '{raw_num}'...")
            
            l_chars = list(raw_num)
            es_valido = False
            try:
                if list(prolog.query(f"validar_celular({l_chars})")):
                    es_valido = True
            except: pass
            
            if es_valido:
                transicion("verificar_numero") # Prolog pasa a 'validando_numero'
                print("✅ NÚMERO REGISTRADO CORRECTAMENTE.")
                print("📩 Te mantendremos informado de las novedades.")
                transicion("finalizar") # Prolog pasa a 'inicio'
                
                print("🤖 Bot: Sesión finalizada. ¡Hasta luego!")
            else:
                print("❌ NÚMERO INVÁLIDO.")
                print("   Recuerda: Código de área + Número = 10 dígitos.")
                print("   Inténtalo de nuevo: ")

if __name__ == "__main__":
    main()

📂 Cargando base de conocimientos...
✅ Sistema Experto online: 13352 vehículos.
📊 Clasificación CSV: {'pickup': 1738, 'auto': 9824, 'suv': 775, 'utilitario': 1015}

                 🚗 CARBOT 🚗
 Sistema Experto | Turing | Automatas | TSP (GPS)
 Escribe 'salir' para terminar.

🤖 Bot: ¡Hola! Bienvenido a la Concesionaria CarBot.
   Selecciona una opción:
   [1] 🚗 Comprar un Auto (Ventas)
   [2] 🔧 Service / Taller (Post-Venta)
⚠️ Opción no válida. Escribe 1 o 2.
🤖 Bot: ¡Hola! Bienvenido a la Concesionaria CarBot.
   Selecciona una opción:
   [1] 🚗 Comprar un Auto (Ventas)
   [2] 🔧 Service / Taller (Post-Venta)
🤖 Bot: ¡Genial! Vamos a buscar tu próximo vehículo.
   Selecciona la categoría:
   [1] 🚗 Auto (Sedan / Hatchback / Coupe)
   [2] 🚙 SUV
   [3] 🛻 Pickup / Camioneta
   [4] 🚐 Utilitario / Furgón
🤖 Bot: Entendido, buscas 'AUTO'.
   ¿Cuál es el precio MÍNIMO que quieres pagar (en USD)?
🤖 Bot: Bien (Mín: $10000). ¿Y cuál es tu presupuesto MÁXIMO (en USD)?
🤖 Bot: Analizando mercado...

✅ Enc